In [272]:
# Look The Note Below!!!

import numpy as np
import time

N = 8 # You shoule change the test size here !!!

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

# Read the note First ! ! !
#### In all the code above, only two of the function usages are needed. One is is_valid(self, state), which is to determine if the current state is legal; the other is is_satisfy(self, state), which is to determine if the current board meets the win condition. 
#### The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.
#### In the first line of the code you can see N = 5, this is the size of the test.
#### Then in the test_block that follows, render indicates whether to use a graphical interface representation, and then method indicates which bts are used(bts or improving_bts).

### Request 1: You should complete the function bts(problem). 
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. And you can add any function you want. (**DDL: 3.31**)
### Request 2: You should complete the function improving_bts(problem). 
You can select one or more methods of the three methods below(Minimum Remaining Value, Least constraining value, Forward checking), but you should have a good performance **when N = 16 without GUI**. (**DDL: 4.07**)

# BTS: Backtracking search (Request 1, DDL: 3.31)

In [267]:
def get_next(pos):
    (i, j) = pos
    if i == N-1 and j == N-1:
        return (-1, -1)
    j += 1
    if (j == N):
        j = 0
        i += 1
    return (i, j)

def bts(problem):
    next = (0, -1)
    action_stack = []
    while not problem.is_satisfy(action_stack):
        if (next != (-1, -1)):
            action_stack.append(next)
            next = (-1, -1)
        flag = False
        (i, j) = action_stack[-1]
        while True:
            (i, j) = get_next((i, j))
            if (i, j) == (-1, -1):
                break
            if (i, j) in action_stack:
                continue
            action_stack[-1] = (i, j)
            if (problem.is_valid(action_stack)):
                next = (0, -1)
                flag = True
                break
        if not flag:
            action_stack.pop()
        if not action_stack:
            print("No solution")
            break
        yield action_stack

# Improving BTS To DO (Request 2, DDL: 4.07)
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [262]:

def improving_bts(problem):

    def dfs(step):
        if (step == N):
            return True
        nonlocal column, diagnoal, action_stack
        for i in range(N):
            if column[i] or diagnoal[0][step+i] or diagnoal[1][step-i+N-1]:
                continue
            column[i] = diagnoal[0][step+i] = diagnoal[1][step-i+N-1] = True
            action_stack.append((step, i))
            if (dfs(step+1)):
                return True
            action_stack.pop()
            column[i] = diagnoal[0][step+i] = diagnoal[1][step-i+N-1] = False
        return False
    
    column = [False for i in range(N)]
    diagnoal = [[False for i in range(N*2-1)] for i in range(2)]
    action_stack = []
    if not dfs(0):
        print("No Solution")
    yield action_stack
    
    # def set_cons(step, pos):
    #     nonlocal board, rest
    #     x, y = pos[0], pos[1]
    #     if (board[x][y] == -1):
    #         board[x][y] = step
    #         rest[x] = (rest[x][0]-1, x)
    
    # def unset_cons(step, pos):
    #     nonlocal board, rest
    #     x, y = pos[0], pos[1]
    #     if (board[x][y] == step):
    #         board[x][y] = -1
    #         rest[x] = (rest[x][0]+1, x)
    
    # def update(step, pos):
    #     nonlocal action_stack, board, rest, setted
    #     x, y = pos[0], pos[1]
    #     action_stack.append(pos)
    #     setted.append(x)
    #     for i in range(N):
    #         set_cons(step, (x, i))
    #         set_cons(step, (i, y))
    #     dx, dy = x, y
    #     while (True):
    #         dx += 1
    #         dy += 1
    #         if dx == N or dy == N:
    #             break
    #         set_cons(step, (dx, dy))
    #     dx, dy = x, y
    #     while (True):
    #         dx -= 1
    #         dy -= 1
    #         if dx == -1 or dy == -1:
    #             break
    #         set_cons(step, (dx, dy))
    #     dx, dy = x, y
    #     while (True):
    #         dx -= 1
    #         dy += 1
    #         if dx == -1 or dy == N:
    #             break
    #         set_cons(step, (dx, dy))
    #     dx, dy = x, y
    #     while (True):
    #         dx += 1
    #         dy -= 1
    #         if dx == N or dy == -1:
    #             break
    #         set_cons(step, (dx, dy))
    
    # def recover(step, pos):
    #     nonlocal action_stack, board, rest
    #     x, y = pos[0], pos[1]
    #     action_stack.pop()
    #     setted.pop()
    #     for i in range(N):
    #         unset_cons(step, (x, i))
    #         unset_cons(step, (i, y))
    #     dx, dy = x, y
    #     while (True):
    #         dx += 1
    #         dy += 1
    #         if dx == N or dy == N:
    #             break
    #         unset_cons(step, (dx, dy))
    #     dx, dy = x, y
    #     while (True):
    #         dx -= 1
    #         dy -= 1
    #         if dx == -1 or dy == -1:
    #             break
    #         unset_cons(step, (dx, dy))
    #     dx, dy = x, y
    #     while (True):
    #         dx -= 1
    #         dy += 1
    #         if dx == -1 or dy == N:
    #             break
    #         unset_cons(step, (dx, dy))
    #     dx, dy = x, y
    #     while (True):
    #         dx += 1
    #         dy -= 1
    #         if dx == N or dy == -1:
    #             break
    #         unset_cons(step, (dx, dy))
    
    # def search(step):
    #     # print("step=", step)
    #     if step == N:
    #         return True
    #     nonlocal action_stack, board, rest, setted
    #     order = sorted(rest)
    #     for i in range(N):
    #         if order[i][1] in setted:
    #             continue
    #         if order[i][0] == 0:
    #             return False
    #         # find a valid row with the least possible assignments
    #         for j in range(N):
    #             if board[i][j] != -1:
    #                 continue
    #             update(step, (i, j))
    #             if (search(step+1)):
    #                 return True
    #             recover(step, (i, j))
    #     return False
    
    # setted = []
    # action_stack = []
    # board = [[-1 for i in range(N)] for j in range(N)]
    # rest = [(N, i) for i in range(N)]
    # if not search(0):
    #     print("No Solution!")
    # print(action_stack)
    # yield action_stack

In [273]:
# test_block
n = N # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
render = False # here to select GUI or not
method = bts  # here to change your method; bts or improving_bts
p = Problem(n)
if render:
    import pygame
    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = method(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen.png')
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in method(p):
        pass
    p.print_state(actions)
    print(time.time() - start_time)


_________________
|Q|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|Q|·|·|
|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|Q|·|
|·|Q|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|
-----------------
82.12266182899475
